In [ ]:
import sys                                                                                                                                                                                                              
sys.path[0]='/home/jovyan/mnt/neuronunit'                                                                                                                                                                                                     
from neuronunit.tests import get_neab                                                                                                                                                                                                         
import neuronunit                                                                                                                                                                                                                             
print(neuronunit.tests.__file__)                                                                                                                                                                                                              
from neuronunit.tests import utilities as outils                                                                                                                                                                                              
                                                                                                                                                                                                                                              
from neuronunit.tests import model_parameters as modelp                                                                                                                                                                                       
import numpy as np                                                                                                                                                                                                                            
model = outils.model                



Getting Rheobase cached data value for from AIBS dataset 354190013
attempting to recover from pickled file
/home/jovyan/mnt/neuronunit/neuronunit/tests/__init__.py
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
Mechanisms already loaded from path: /home/jovyan/mnt/neuronunit/neuronunit/tests/NeuroML2.  Aborting.
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml


In [ ]:

class Score:
    def __init__(self, score):
        self.score = score

class Test:
    def _optimize(self, model,modelp):
        '''
        The implementation of optimization, consisting of implementation details.
        Inputs a model, and model parameter ranges to expore
        Private method for programmer designer.
        Outputs the optimal model, its attributes and the low error it resulted in.
        '''
        from neuronunit.tests import nsga
        gap = nsga.GAparams(model)
        # The number of generations is 2
        gap.NGEN = 2
        # The population of genes is 4
        gap.MU = 4
        
        gap.BOUND_LOW = [ np.min(i) for i in modelp.model_params.values() ]
        gap.BOUND_UP = [ np.max(i) for i in modelp.model_params.values() ]

        vmpop, pop, invalid_ind, pf = nsga.main(gap.NGEN,gap.MU,model,modelp)
        some_tuples = (vmpop, pop, invalid_ind, pf)
        return pop[0], some_tuples
        #return vmpop, pop, invalid_ind, pf
        
    def _get_optimization_parameters(self, some_tuples):
        vmpop = some_tuples[0]
        # Your specific unpacking of tuples that _optimize returns
        scores = vmpop[0].score
        pop = some_tuples[1]
        errors = pop[0].fitness
        parameters = vmpop[0].attrs
        #parameters,errors,scores,_ = zip(*some_tuples)
        return parameters,scores

    def optimize(self, model, modelp):
        '''
        The Class users version of optimize
        where details are hidden in _optimizae
        '''

        # Do optimization including repeated calls to judge
        model, some_tuples = self._optimize(model,modelp)
        parameters, scores = self._get_optimization_parameters(some_tuples)
        # Maybe rebuild the original model
        # (i.e. restore the true class from the virtual version)

        # Your code keeps parameter sets and associated scores
        # All the organizing stuff
        
        
        # this a way of looking at solved model parameters, ie candidate solutions from 
        # the pareto front.

        
        # Make a ScoreArray (which is basically a pandas dataframe)
        path = sciunit.ScoreArray(models,scores=scores)
        return model, path, some_tuples

    
t = Test()


model,path,some_tuples = t.optimize(model,modelp)   

<module 'neuronunit.tests.model_parameters' from '/home/jovyan/mnt/neuronunit/neuronunit/tests/model_parameters.py'>
0 0.00133124041587
1 -4.62650322561e-09
2 -40.5489616705
3 0.193492966463
4 33.2185458611
5 0.120618883919
6 -45.9853381238
7 -58.6430535334
8 -64.8537049938
9 0.000107714942576
0 0.00122496518895
1 -4.12559166363e-09
2 -43.5671413225
3 0.0557320175721
4 30.6352604888
5 0.486215065799
6 -65.0502489357
7 -57.6358779931
8

/opt/conda/lib/python3.5/site-packages/scoop-0.7.2.0-py3.5.egg/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().


 -72.4674901551
9 9.89513607484e-05
0 0.000653747062718
1 -4.99673150457e-09
2 -47.8961117794
3 0.118477226439
4 33.1803773429
5 0.720501999015
6 -45.1166327844
7 -57.183713787
8 -58.2960986661
9 9.7582070666e-05
0 0.000919196194865
1 -3.72615056711e-09
2 -39.9123594756
3 0.190154253926
4 36.4823756252
5 0.166655408226
6 -46.5212568888
7 -57.0249193462
8 -71.7930714896
9 9.54032431356e-05
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
<class 'float'>
current 196.75925925925927 spikes 1
{}
<class 'float'>
current 196.875 spikes 1
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
<class 'float'>
current 255.90277777777777 spikes 1
{}
<class 'float'>
{}


In [ ]:
path
models = [model.__class__(p) for p in parameters]

    

In [ ]:

import pickle
import pandas as pd


try:
    ground_error = pickle.load(open('big_model_evaulated.pickle','rb'))
except:
    '{0} it seems the error truth data does not yet exist, lets create it now '.format(str(False))
    ground_error = list(futures.map(outils.func2map, ground_truth))
    pickle.dump(ground_error,open('big_model_evaulated.pickle','wb'))
ground_error_nsga=list(zip(vmpop,pop,invalid_ind))
pickle.dump(ground_error_nsga,open('nsga_evaulated.pickle','wb'))

#Get the differences between values obtained via brute force, and those obtained otherwise:
sum_errors = [ i[0] for i in ground_error ]
composite_errors = [ i[1] for i in ground_error ]
attrs = [ i[2] for i in ground_error ]
rheobase = [ i[3] for i in ground_error ]

indexs = [i for i,j in enumerate(sum_errors) if j==np.min(sum_errors) ][0]
indexc = [i for i,j in enumerate(composite_errors) if j==np.min(composite_errors) ][0]
#assert indexs == indexc
vmpop = some_tupeles[0]
df_0 = pd.DataFrame([ (k,v,vmpop[0].attrs[k],float(v)-float(vmpop[0].attrs[k])) for k,v in ground_error[indexc][2].items() ])
df_1 = pd.DataFrame([ (k,v,vmpop[1].attrs[k],float(v)-float(vmpop[1].attrs[k])) for k,v in ground_error[indexc][2].items() ])



In [ ]:


df_0

In [ ]:
df_1
